In [1]:
import os
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from tensorflow.keras.models import load_model
from keras.applications.efficientnet import preprocess_input
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path

root_dir = Path().resolve().parents[0]
print("root_dir")
# LOAD THE TRAINED MODEL
model_pth = os.path.join(root_dir, "src/model/")
model = load_model(model_pth)
print("FUSED-DENSENET-TINY INITIALIZED!")
print("The model consists of", len(model.layers), "layers")

# CHOOSE TEST DATASET
test_data_dir = "C:\\Users\\HP\\OneDrive\\Desktop\\DFDC_DATASET\\test"
# PREPARE THE DATA INPUT
img_width, img_height = 128,128
img_rows, img_cols = 128, 128
input_shape = (img_rows,img_cols,3)

# DATA GENERATORS
batch_size = 16
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=False,
classes=['fake', 'real']
)


# CHECK THE NUMBER OF SAMPLES
nb_test_samples = len(test_generator.filenames)
if nb_test_samples == 0:
  print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
  print("Check the data folders first!")
else:
  print("Validation samples found!")
# CHECK THE CLASS INDICES
print(test_generator.class_indices)
# TRUE LABELS
Y_test = test_generator.classes
num_classes = len(test_generator.class_indices)
if nb_test_samples > 0:
  print("Generators are set!")
  print("Check if the dataset is complete and has no problems before proceeding.")

# Make predictions
y_pred = model.predict(test_generator, nb_test_samples/batch_size, workers=1)


#Evaluate model
accuracy = accuracy_score(Y_test, y_pred.argmax(axis=-1))
print('The accuracy of the model is:', accuracy)
fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred.argmax(axis=-1))
auc = metrics.auc(fpr, tpr)
print('AUC:', auc)
# PRINT CONFUSION MATRIX
# print(confusion_matrix(Y_test, y_pred.argmax(axis=-1)))
conf = confusion_matrix(Y_test, y_pred.argmax(axis=-1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf)
disp.plot()
plt.savefig('C:\\Users\\HP\\OneDrive\\Desktop\\DFDC_DATASET\\testing result', dpi=300)



root_dir


ValueError: File format not supported: filepath=D:\Transfer-Learning-of-DenseNet-Model-for-Deep-Fake-Detection\src\src/model/. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(D:\Transfer-Learning-of-DenseNet-Model-for-Deep-Fake-Detection\src\src/model/, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [2]:
import time
from tkinter.tix import InputOnly
# from wsgiref.types import InputStream
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D, Dense, Add, Dropout
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, classification_report, confusion_matrix, roc_curve, auc
from pathlib import Path
import os

C:\Users\DELL\AppData\Local\Temp\ipykernel_15968\3354300503.py:2: DeprecationWarning: The Tix Tk extension is unmaintained, and the tkinter.tix wrapper module is deprecated in favor of tkinter.ttk
  from tkinter.tix import InputOnly


In [4]:
Path().resolve().parents[0]

WindowsPath('D:/Transfer-Learning-of-DenseNet-Model-for-Deep-Fake-Detection/src')

In [11]:
root_dir = Path().resolve().parents[1]
root_dir

WindowsPath('D:/Transfer-Learning-of-DenseNet-Model-for-Deep-Fake-Detection')

In [3]:
# Define the paths for train and validation data directories
train_data_dir = os.path.join(root_dir,'data/train')
validation_data_dir = os.path.join(root_dir,'data/validation')

In [4]:
# Set the input image dimensions
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)
model_input = Input(shape=input_shape)

#model_input = InputOnly(shape=input_shape)
print("The Input size is set to", model_input)

The Input size is set to <KerasTensor shape=(None, 128, 128, 3), dtype=float32, sparse=None, name=keras_tensor>


In [5]:
# Data Generators
batch_size = 16
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['fake', 'real']
)
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=['fake', 'real']
)

Found 36075 images belonging to 2 classes.
Found 18621 images belonging to 2 classes.


In [6]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)

In [7]:
# Check if the training samples are found
if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")

# Check if the validation samples are found
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")

Train samples found!
Validation samples found!


In [8]:
# Check the class indices
train_generator.class_indices
validation_generator.class_indices
num_classes = len(train_generator.class_indices)
if nb_train_samples and nb_validation_samples > 0:
    print("Generators are set!")
    print("Check if the dataset is complete and has no problems before proceeding.")

Generators are set!
Check if the dataset is complete and has no problems before proceeding.


In [9]:
# # Transfer Learning - DenseNet121-A
# def densenet_tiny_A_builder(model_input):
#     densenet_tiny_A_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
#     # Partial Layer Freezing
#     for layer in densenet_tiny_A_builder.layers:
#         layer.trainable = False
#     # Model Truncation
#     x = densenet_tiny_A_builder.layers[-354].output
#     model = Model(inputs=densenet_tiny_A_builder.input, outputs=x, name='densenet-tiny-A')
#     return model


In [10]:
# # Generate the DenseNet121-A model
# densenet_tiny_A = densenet_tiny_A_builder(model_input)

# # Plot the model structure
# print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
# densenet_tiny_A.summary()
# print("DenseNet-Tiny-A successfully built!")

In [11]:
# # Transfer Learning - DenseNet121-B
# def densenet_tiny_B_builder(model_input):
#     densenet_tiny_B_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
#     # Re-training all layers (re-naming layers to prevent overlaps)
#     for layer in densenet_tiny_B_builder.layers:
#         layer.trainable = True
#         # layer._name = layer._name + str("_mirror")
#     # Model Truncation
#     x = densenet_tiny_B_builder.layers[-354].output
#     model = Model(inputs=densenet_tiny_B_builder.input, outputs=x, name='densenet_tiny-B')
#     return model

In [12]:
# # Generate the DenseNet121-B model
# densenet_tiny_B = densenet_tiny_B_builder(model_input)

# # Plot the model structure
# print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
# densenet_tiny_B.summary()
# print("DenseNet-Tiny-B successfully built!")

In [13]:
# # Transfer Learning - DenseNet121-A
# def densenet_tiny_A_builder(model_input):
#     densenet_tiny_A_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
#     # Partial Layer Freezing
#     for layer in densenet_tiny_A_builder.layers:
#         layer.trainable = False
#     # Model Truncation
#     x = densenet_tiny_A_builder.layers[-354].output
#     model = Model(inputs=densenet_tiny_A_builder.input, outputs=x, name='densenet-tiny-A')
#     return model

# # Generate the DenseNet121-A model
# densenet_tiny_A = densenet_tiny_A_builder(model_input)

# # Plot the model structure
# print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
# densenet_tiny_A.summary()
# print("DenseNet-Tiny-A successfully built!")    

# # Transfer Learning - DenseNet121-B
# def densenet_tiny_B_builder(model_input):
#     densenet_tiny_B_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
#     # Re-training all layers (re-naming layers to prevent overlaps)
#     for layer in densenet_tiny_B_builder.layers:
#         layer.trainable = True
#         # layer._name = layer._name + str("_mirror")
#     # Model Truncation
#     x = densenet_tiny_B_builder.layers[-354].output
#     model = Model(inputs=densenet_tiny_B_builder.input, outputs=x, name='densenet_tiny-B')
#     return model

# # Generate the DenseNet121-B model
# densenet_tiny_B = densenet_tiny_B_builder(model_input)

# # Plot the model structure
# print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
# densenet_tiny_B.summary()
# print("DenseNet-Tiny-B successfully built!")


### GPT Corrected

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.applications import DenseNet121

# Transfer Learning - DenseNet121-A
def densenet_tiny_A_builder(input_shape):
    model_input = Input(shape=input_shape)
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Partial Layer Freezing
    for layer in base_model.layers:
        layer.trainable = False
    # Model Truncation
    x = base_model.layers[-354].output
    model = Model(inputs=base_model.input, outputs=x, name='densenet-tiny-A')
    return model

# Transfer Learning - DenseNet121-B
def densenet_tiny_B_builder(input_shape):
    model_input = Input(shape=input_shape)
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Re-training all layers (renaming layers to prevent overlaps if needed)
    for layer in base_model.layers:
        layer.trainable = True
        # layer._name = layer._name + '_mirror'  # Uncomment if renaming is necessary
    # Model Truncation
    x = base_model.layers[-354].output
    model = Model(inputs=base_model.input, outputs=x, name='densenet_tiny-B')
    return model

## Define input shape
# input_shape = (224, 224, 3)

img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)

# Generate the DenseNet121-A model
densenet_tiny_A = densenet_tiny_A_builder(input_shape)

# Plot the model structure
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
densenet_tiny_A.summary()
print("DenseNet-Tiny-A successfully built!")    

# Generate the DenseNet121-B model
densenet_tiny_B = densenet_tiny_B_builder(input_shape)

# Plot the model structure
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
densenet_tiny_B.summary()
print("DenseNet-Tiny-B successfully built!")


PLEASE CHECK THE ENTIRE MODEL UP TO THE END


Model: "densenet-tiny-A"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 134, 134,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 32, 32,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 32, 32,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 32, 32,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 32, 32,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 32, 32,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 32, 32,    │     12,288 │ conv2_block2_0_r

 Total params: 557,248 (2.13 MB)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 134, 134,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 32, 32,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 32, 32,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 32, 32,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 32, 32,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 32, 32,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 32, 32,    │     12,288 │ conv2_block2_0_r

 Total params: 557,248 (2.13 MB)

 Trainable params: 551,616 (2.10 MB)

 Non-trainable params: 5,632 (22.00 KB)

DenseNet-Tiny-B successfully built!


In [14]:
# Concatenate the models as a single pipeline
models = [densenet_tiny_A, densenet_tiny_B]
print("Concatenation success!")
print("Fused-DenseNet-Tiny ready to connect with its ending layers!")

Concatenation success!
Fused-DenseNet-Tiny ready to connect with its ending layers!


In [15]:
# Build the Fused-DenseNet-Tiny
def fused_densenet_tiny(models, model_input):
    outputs = [m.output for m in models]
    y = Add()(outputs)
    y = GlobalAveragePooling2D()(y)
    y = Dense(512, activation='relu', use_bias=True)(y)
    y = Dropout(0.5)(y)
    prediction = Dense(num_classes, activation='softmax', name='Softmax_Classifier')(y)
    model = Model(model_input, prediction, name='fused_densenet_tiny')
    return model

In [16]:
# Instantiate the ensemble model and report the summary
fused_densenet_tiny = fused_densenet_tiny(models, model_input)
print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print("Fused-DenseNet-Tiny complete and ready for compilation and training!")
print()
print()
print()
fused_densenet_tiny.summary()
print("Building of the Fused-DenseNet-Tiny COMPLETE!")

ValueError: The name "conv1_conv" is used 2 times in the model. All operation names should be unique.

In [19]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Concatenate
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import clone_model

# Transfer Learning - DenseNet121-A
def densenet_tiny_A_builder(input_shape):
    model_input = Input(shape=input_shape)
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Partial Layer Freezing
    for layer in base_model.layers:
        layer.trainable = False
    # Model Truncation
    x = base_model.layers[-354].output
    model = Model(inputs=base_model.input, outputs=x, name='densenet-tiny-A')
    return model

# Transfer Learning - DenseNet121-B
def densenet_tiny_B_builder(input_shape):
    model_input = Input(shape=input_shape)
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Re-training all layers (renaming layers to prevent overlaps if needed)
    for layer in base_model.layers:
        layer.trainable = True
    # Model Truncation
    x = base_model.layers[-354].output
    model = Model(inputs=base_model.input, outputs=x, name='densenet_tiny-B')
    return model

# Define input shape
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)

# Generate the DenseNet121-A model
densenet_tiny_A = densenet_tiny_A_builder(input_shape)

# Generate the DenseNet121-B model
densenet_tiny_B = densenet_tiny_B_builder(input_shape)

# Cloning models to avoid naming conflicts
def clone_and_rename(model, suffix):
    cloned_model = clone_model(model)
    cloned_model.set_weights(model.get_weights())
    for layer in cloned_model.layers:
        layer._name = layer.name + suffix
    return cloned_model

# Clone models with unique suffixes
cloned_model_A = clone_and_rename(densenet_tiny_A, "_A")
cloned_model_B = clone_and_rename(densenet_tiny_B, "_B")

# Combine models
def fused_densenet_tiny(models, input_shape, num_classes=10):
    model_input = Input(shape=input_shape)
    outputs = [model(model_input) for model in models]
    merged = Concatenate()(outputs)
    y = Dense(256, activation='relu')(merged)
    y = Dropout(0.5)(y)
    y = Dense(128, activation='relu')(y)
    prediction = Dense(num_classes, activation='softmax', name='Softmax_Classifier')(y)
    model = Model(inputs=model_input, outputs=prediction, name='fused_densenet_tiny')
    return model

# List of cloned models
models = [cloned_model_A, cloned_model_B]

# Instantiate the ensemble model
fused_densenet_tiny_model = fused_densenet_tiny(models, input_shape)

# Print the model summary
print("PLEASE CHECK THE MODEL UP TO THE END")
print("Fused-DenseNet-Tiny complete and ready for compilation and training!")
fused_densenet_tiny_model.summary()
print("Building of the Fused-DenseNet-Tiny COMPLETE!")


AttributeError: Exception encountered when calling Concatenate.call().

[1m'list' object has no attribute 'shape'[0m

Arguments received by Concatenate.call():
  • args=([['<KerasTensor shape=(None, 16, 16, 224), dtype=float32, sparse=False, name=keras_tensor_4359>'], ['<KerasTensor shape=(None, 16, 16, 224), dtype=float32, sparse=False, name=keras_tensor_4360>']],)
  • kwargs=<class 'inspect._empty'>

In [23]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.applications.densenet import DenseNet121, preprocess_input
# from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D, Dense, Add, Dropout
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, classification_report, confusion_matrix, roc_curve, auc
from pathlib import Path


import time
# from wsgiref.types import InputStream
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D, Dense, Add, Dropout
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, classification_report, confusion_matrix, roc_curve, auc
from pathlib import Path
import os
# Define the paths for train and validation data directories


# Set the input image dimensions
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)
model_input = Input(shape=input_shape)

print("The Input size is set to", model_input)

# Data Generators
batch_size = 16
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['fake', 'real']
)
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=['fake', 'real']
)
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
print("Data folders found!")

# Check if the training samples are found
if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")

# Check if the validation samples are found
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")

# Check the class indices
train_generator.class_indices
validation_generator.class_indices
num_classes = len(train_generator.class_indices)
if nb_train_samples and nb_validation_samples > 0:
    print("Generators are set!")
    print("Check if the dataset is complete and has no problems before proceeding.")

# Utility function to rename layers
def rename_layers(model, suffix):
    for layer in model.layers:
        layer._name = layer.name + suffix
    return model

# Transfer Learning - DenseNet121-A
def densenet_tiny_A_builder(model_input):
    densenet_tiny_A = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Partial Layer Freezing
    for layer in densenet_tiny_A.layers:
        layer.trainable = False
    # Model Truncation
    x = densenet_tiny_A.layers[-1].output
    model = Model(inputs=densenet_tiny_A.input, outputs=x, name='densenet-tiny-A')
    return model

# Generate the DenseNet121-A model
densenet_tiny_A = densenet_tiny_A_builder(model_input)

# Plot the model structure
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
densenet_tiny_A.summary()
print("DenseNet-Tiny-A successfully built!")

# Transfer Learning - DenseNet121-B
def densenet_tiny_B_builder(model_input):
    densenet_tiny_B = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    # Re-training all layers (renaming layers to ensure uniqueness)
    densenet_tiny_B = rename_layers(densenet_tiny_B, "_mirror")
    for layer in densenet_tiny_B.layers:
        layer.trainable = True
    # Model Truncation
    x = densenet_tiny_B.layers[-1].output
    model = Model(inputs=densenet_tiny_B.input, outputs=x, name='densenet-tiny-B')
    return model

# Generate the DenseNet121-B model
densenet_tiny_B = densenet_tiny_B_builder(model_input)

# Plot the model structure
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
densenet_tiny_B.summary()
print("DenseNet-Tiny-B successfully built!")

# Concatenate the models as a single pipeline
models = [densenet_tiny_A, densenet_tiny_B]
print("Concatenation success!")
print("Fused-DenseNet-Tiny ready to connect with its ending layers!")

# Build the Fused-DenseNet-Tiny
def fused_densenet_tiny(models, model_input):
    outputs = [m.output for m in models]
    y = Add()(outputs)
    y = GlobalAveragePooling2D()(y)
    y = Dense(512, activation='relu', use_bias=True)(y)
    y = Dropout(0.5)(y)
    prediction = Dense(num_classes, activation='softmax', name='Softmax_Classifier')(y)
    model = Model(model_input, prediction, name='fused_densenet_tiny')
    return model

# Instantiate the ensemble model and report the summary
fused_densenet_tiny_model = fused_densenet_tiny(models, model_input)
print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print("Fused-DenseNet-Tiny complete and ready for compilation and training!")
print()
print()
print()
fused_densenet_tiny_model.summary()
print("Building of the Fused-DenseNet-Tiny COMPLETE!")

batch_size = 16
epochs = 10
start_time = time.time()
fused_densenet_tiny_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy']) 

callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    mode="max",
    restore_best_weights=True,
)

history = fused_densenet_tiny_model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1
)

elapsed_time = time.time() - start_time
print("Elapsed time:", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

# Save the Fused-DenseNet-Tiny after training completes
fused_densenet_tiny_model.save("C:\\Users\\HP\\OneDrive\\Desktop\\DFDC_DATASET\\results")

train_accuracy = history.history['accuracy'][-1]
print('Overall Training Accuracy:', train_accuracy)

The Input size is set to <KerasTensor shape=(None, 128, 128, 3), dtype=float32, sparse=None, name=keras_tensor_5219>
Found 36075 images belonging to 2 classes.
Found 18621 images belonging to 2 classes.
Data folders found!
Train samples found!
Validation samples found!
Generators are set!
Check if the dataset is complete and has no problems before proceeding.
PLEASE CHECK THE ENTIRE MODEL UP TO THE END


Model: "densenet-tiny-A"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_20   │ (None, 134, 134,  │          0 │ input_layer_13[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_21   │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 32, 32,    │          0 │ zero_padding2d_2… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 32, 32,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 32, 32,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 32, 32,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 32, 32,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 32, 32,    │     12,288 │ conv2_block2_0_r

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

...

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 6,953,856 (26.53 MB)

 Non-trainable params: 83,648 (326.75 KB)

DenseNet-Tiny-B successfully built!
Concatenation success!
Fused-DenseNet-Tiny ready to connect with its ending layers!


ValueError: The name "conv1_conv" is used 2 times in the model. All operation names should be unique.

In [25]:
import os
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from tensorflow.keras.models import load_model
from keras.applications.efficientnet import preprocess_input
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# LOAD THE TRAINED MODEL
model = load_model(os.path.join(root_dir, "data/results/model.h5"))
print("FUSED-DENSENET-TINY INITIALIZED!")
print("The model consists of", len(model.layers), "layers")

# CHOOSE TEST DATASET
test_data_dir = os.path.join(root_dir, "data/test")
# PREPARE THE DATA INPUT
img_width, img_height = 128,128
img_rows, img_cols = 128, 128
input_shape = (img_rows,img_cols,3)

# DATA GENERATORS
batch_size = 16
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=False,
classes=['fake', 'real']
)


# CHECK THE NUMBER OF SAMPLES
nb_test_samples = len(test_generator.filenames)
if nb_test_samples == 0:
  print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
  print("Check the data folders first!")
else:
  print("Validation samples found!")
# CHECK THE CLASS INDICES
print(test_generator.class_indices)
# TRUE LABELS
Y_test = test_generator.classes
num_classes = len(test_generator.class_indices)
if nb_test_samples > 0:
  print("Generators are set!")
  print("Check if the dataset is complete and has no problems before proceeding.")

# Make predictions
y_pred = model.predict(test_generator, nb_test_samples/batch_size, workers=1)


#Evaluate model
accuracy = accuracy_score(Y_test, y_pred.argmax(axis=-1))
print('The accuracy of the model is:', accuracy)
fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred.argmax(axis=-1))
auc = metrics.auc(fpr, tpr)
print('AUC:', auc)
# PRINT CONFUSION MATRIX
# print(confusion_matrix(Y_test, y_pred.argmax(axis=-1)))
conf = confusion_matrix(Y_test, y_pred.argmax(axis=-1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf)
disp.plot()
# plt.savefig('C:\\Users\\HP\\OneDrive\\Desktop\\DFDC_DATASET\\testing result', dpi=300)



TypeError: Error when deserializing class 'Conv2D' using config={'name': 'conv1/conv', 'trainable': False, 'dtype': 'float32', 'filters': 64, 'kernel_size': [7, 7], 'strides': [2, 2], 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Argument `name` must be a string and cannot contain character `/`. Received: name=conv1/conv (of type <class 'str'>)

In [26]:
import json
import os
import tensorflow as tf
from keras.models import model_from_json

# Define the path to your model
model_path = os.path.join(root_dir, "data/results/model.h5")

# Load the model
model = tf.keras.models.load_model(model_path)

# Get model configuration in JSON format
model_config = model.to_json()

# Parse the JSON string to a dictionary
model_config_dict = json.loads(model_config)

# Function to recursively fix layer names
def fix_layer_names(config_dict):
    if isinstance(config_dict, dict):
        if 'name' in config_dict and '/' in config_dict['name']:
            config_dict['name'] = config_dict['name'].replace('/', '_')
        for key, value in config_dict.items():
            fix_layer_names(value)
    elif isinstance(config_dict, list):
        for item in config_dict:
            fix_layer_names(item)

# Fix layer names in the model configuration
fix_layer_names(model_config_dict)

# Convert the fixed configuration back to JSON string
fixed_model_config = json.dumps(model_config_dict)

# Load the model from the fixed configuration
model = model_from_json(fixed_model_config)
model.load_weights(model_path)

print("FUSED-DENSENET-TINY INITIALIZED!")
print("The model consists of", len(model.layers), "layers")

# Continue with the rest of your code...


TypeError: Error when deserializing class 'Conv2D' using config={'name': 'conv1/conv', 'trainable': False, 'dtype': 'float32', 'filters': 64, 'kernel_size': [7, 7], 'strides': [2, 2], 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Argument `name` must be a string and cannot contain character `/`. Received: name=conv1/conv (of type <class 'str'>)

## Testing 

In [13]:
import os
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from tensorflow.keras.models import load_model
from keras.applications.efficientnet import preprocess_input
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path

root_dir = Path().resolve().parents[1]
print('\n\n' , root_dir)



 D:\Transfer-Learning-of-DenseNet-Model-for-Deep-Fake-Detection


In [15]:
# LOAD THE TRAINED MODEL
model_pth = os.path.join(root_dir, "src/model/")
model = tf.saved_model.load(model_pth)
print("FUSED-DENSENET-TINY INITIALIZED!")
# print("The model consists of", len(model.layers), "layers")

FUSED-DENSENET-TINY INITIALIZED!


In [16]:
test_data_dir = os.path.join(root_dir, "data/test")
# PREPARE THE DATA INPUT
img_width, img_height = 128,128
img_rows, img_cols = 128, 128
input_shape = (img_rows,img_cols,3)

# DATA GENERATORS
batch_size = 16
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=False,
classes=['fake', 'real']
)


# CHECK THE NUMBER OF SAMPLES
nb_test_samples = len(test_generator.filenames)
if nb_test_samples == 0:
  print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
  print("Check the data folders first!")
else:
  print("Validation samples found!")
# CHECK THE CLASS INDICES
print(test_generator.class_indices)
# TRUE LABELS
Y_test = test_generator.classes
num_classes = len(test_generator.class_indices)
if nb_test_samples > 0:
  print("Generators are set!")
  print("Check if the dataset is complete and has no problems before proceeding.")

# Make predictions
y_pred = model.predict(test_generator, nb_test_samples/batch_size, workers=1)


#Evaluate model
accuracy = accuracy_score(Y_test, y_pred.argmax(axis=-1))
print('The accuracy of the model is:', accuracy)
fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred.argmax(axis=-1))
auc = metrics.auc(fpr, tpr)
print('AUC:', auc)
# PRINT CONFUSION MATRIX
# print(confusion_matrix(Y_test, y_pred.argmax(axis=-1)))
conf = confusion_matrix(Y_test, y_pred.argmax(axis=-1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf)
disp.plot()
# plt.savefig('C:\\Users\\HP\\OneDrive\\Desktop\\DFDC_DATASET\\testing result', dpi=300)



Found 25790 images belonging to 2 classes.
Validation samples found!
{'fake': 0, 'real': 1}
Generators are set!
Check if the dataset is complete and has no problems before proceeding.


AttributeError: '_UserObject' object has no attribute 'predict'

In [21]:
from fastapi import FastAPI
import tensorflow as tf
import os
from pathlib import Path

app = FastAPI()

# Function to load the frozen graph
def load_frozen_graph(model_path):
    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found at {model_path}")
    
    # Load the frozen graph
    with tf.io.gfile.GFile(str(model_path), "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    
    # Import the graph
    with tf.compat.v1.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    
    return graph

@app.on_event("startup")
async def startup_event():
    global graph, session, input_tensor, output_tensor

    # Load the graph
    graph = load_frozen_graph(MODEL_PATH)

    # Create a session to run the graph
    session = tf.compat.v1.Session(graph=graph)

    # Identify input and output tensors by their names
    input_tensor = graph.get_tensor_by_name('input_tensor_name:0')
    output_tensor = graph.get_tensor_by_name('output_tensor_name:0')

@app.get("/")
async def read_root():
    return {"message": "Model successfully loaded"}

@app.post("/predict")
async def predict(data: List[List[float]]):
    # Make predictions using the loaded model
    predictions = session.run(output_tensor, feed_dict={input_tensor: data})
    return {"predictions": predictions.tolist()}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


C:\Users\DELL\AppData\Local\Temp\ipykernel_15968\3640679236.py:24: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


NameError: name 'List' is not defined